In [1]:
#爬取起点中文网完本小说信息

In [2]:
import requests,json,time,re
from requests.exceptions import RequestException
from pyquery import PyQuery as pq
from fontTools.ttLib import TTFont
from io import BytesIO
import pandas as pd

In [10]:
start_url = 'https://www.qidian.com/finish?action=hidden&orderId=3&vip=1&sign=1&style=2&pageSize=50&siteid=1&pubflag=0&hiddenField=2&page='

In [5]:
def get_font(url):
    response = requests.get(url)
    font = TTFont(BytesIO(response.content))
    cmap = font.getBestCmap()
    font.close()
    return cmap

In [6]:
def get_encode(cmap,values):
    WORD_MAP = {'zero':'0','one':'1','two':'2','three':'3','four':'4','five':'5','six':'6','seven':'7','eight':'8','nine':'9','period':'.'}
    word_count=''
    for value in values.split(';'):
        value = value[2:]
        key = cmap[int(value)]
        word_count += WORD_MAP[key]
    return word_count

In [7]:
def get_index(start_url):
    #获取当前页面的html
    response = requests.get(start_url).text   
    doc = pq(response)
    #获取当前字体文件名称（point1）
    classattr = doc('tr .total span').attr('class')
    pattern = '</style><span.*?%s.*?>(.*?)</span>'%classattr
    #获取当前页面所有被字数字符
    numberlist = re.findall(pattern,response)
    #获取当前包含字体文件链接的文本（point2）
    fonturl = doc('tr .total style').text() 
    #通过正则获取当前页面字体文件链接
    url = re.search('woff.*?url.*?\'(.+?)\'.*?truetype',fonturl) .group(1)
    cmap = get_font(url)
#     point3
    books = doc('tbody > tr').items()
    i = 0
    df = pd.DataFrame()
    for book in books:
        item = {}
        item['类别'] = book('.td-one').text()
        item['小说名称'] = book('.name').text()
        item['最新章节'] = book('.chapter').text()
        item['字数'] = get_encode(cmap,numberlist[i][:-1])
        item['作者'] = book('.author').text()        
        item['更新时间'] = book('.date').text()
        i += 1
        df = df.append(item, ignore_index=True)
    return df

In [11]:
for page in range(1,196):
    url = start_url + str(page)
    get_index(url)

In [12]:
url = start_url + str(1)
get_index(url)

,作者,字数,小说名称,更新时间,最新章节,类别
0,雷云风暴,2019.35,从零开始,2016-03-21,第23卷 第六章放飞希望（完结篇）,「游戏·虚拟网游」
1,小段探花,1588.76,修神外传,2016-10-09,探花新书《修神外传仙界篇》已经上传,「仙侠·幻想修仙」
2,汉宝,1261.26,移动藏经阁,2018-08-02,番外，普林星系篇（2）,「玄幻·东方玄幻」
3,傅啸尘,1168.08,武神空间,2016-09-17,完本感言！读者请进！,「玄幻·东方玄幻」
4,傲月长空,1104.23,异世傲天,2018-01-15,第1878~1882【大结局！】,「玄幻·异世大陆」
5,卧牛真人,1040.22,修真四万年,2018-08-31,新书预告结束和开始,「科幻·未来世界」
6,天雷猪,1011.16,剑神重生,2014-06-01,完本感言,「玄幻·异世大陆」
7,张扬的五月,994.86,网游之倒行逆施,2018-04-30,完本感言,「游戏·虚拟网游」
8,velver,988.68,左道旁门,2015-12-22,后记,「都市·都市生活」
9,幻雨,978.6,百炼成仙,2015-05-26,幻雨新书《厨道仙途》，大家请去看看哦！,「仙侠·幻想修仙」


In [18]:
df3

,作者,字数,小说名称,更新时间,最新章节,类别
0,雷云风暴,2019.35,从零开始,2016-03-21,第23卷 第六章放飞希望（完结篇）,「游戏·虚拟网游」
1,小段探花,1588.76,修神外传,2016-10-09,探花新书《修神外传仙界篇》已经上传,「仙侠·幻想修仙」
2,汉宝,1261.26,移动藏经阁,2018-08-02,番外，普林星系篇（2）,「玄幻·东方玄幻」
3,傅啸尘,1168.08,武神空间,2016-09-17,完本感言！读者请进！,「玄幻·东方玄幻」
4,傲月长空,1104.23,异世傲天,2018-01-15,第1878~1882【大结局！】,「玄幻·异世大陆」
5,卧牛真人,1040.22,修真四万年,2018-08-31,新书预告结束和开始,「科幻·未来世界」
6,天雷猪,1011.16,剑神重生,2014-06-01,完本感言,「玄幻·异世大陆」
7,张扬的五月,994.86,网游之倒行逆施,2018-04-30,完本感言,「游戏·虚拟网游」
8,velver,988.68,左道旁门,2015-12-22,后记,「都市·都市生活」
9,幻雨,978.6,百炼成仙,2015-05-26,幻雨新书《厨道仙途》，大家请去看看哦！,「仙侠·幻想修仙」


In [19]:
df3.tail()

,作者,字数,小说名称,更新时间,最新章节,类别
45,幽兰飘香,11.18,母爱无悔,2010-07-03,第五章母爱无悔,「短篇·短篇小说」
46,丁东歌,10.96,接头,2010-12-29,第三十三章接头,「短篇·短篇小说」
47,东道士,10.82,斗阵洪荒,2010-09-01,第五十章炎黄定天，八卦立碑【大结局】,「短篇·短篇小说」
48,易子龙,10.7,追与赶,2010-07-13,第二十四话非走不可,「短篇·短篇小说」
49,酌古,10.68,对面男生之阿土哥,2011-06-02,第三十六章执着（大结局）,「短篇·短篇小说」


In [21]:
df3.to_csv("out.csv")